# Freping Demo

In [ ]:
from rtmha.freping import Freping
import numpy as np
import plotly.graph_objects as go
from mlab import generate_sine_waves
import ipywidgets as widgets

In [ ]:
fig = go.FigureWidget()

def do_freping(k=None):
    x, y = generate_sine_waves([freqw.value])
    f = Freping(int(lenw.value), 32, alpha.value, True)
    f_out=[]
    for d in [y[x:x+32] for x in range(0, len(y), 32)]:
        f_out = np.append(f_out, f.proc(d))
    plot_fft(fig, f'Freping {freqw.value}Hz alpha={alpha.value}', y, f_out)

In [ ]:
def plot_fft(fig, title, out1, out2, update=False):
    ftrans = np.abs(np.fft.fft(out1)/len(out1))
    ftrans2 = np.abs(np.fft.fft(out2)/len(out2))
    if not plot_fft.first:
        with fig.batch_update():
            fig.data[0].y = ftrans
            fig.data[1].y = ftrans2
            fig.update_layout(title=title)
            return
        
    plot_fft.first = False
    outlen = len(out1)
    values = np.arange(int(outlen/2))
    period = outlen/32000
    frequencies = values/period

    fig.add_trace(go.Scatter(x=frequencies, line_color='green', opacity=0.4, name='orig', y=ftrans))
    fig.add_trace(go.Scatter(x=frequencies, opacity=0.8, name='frep', y=ftrans2))

    fig.update_layout(title=title,
                    xaxis_title='Frequency',
                    yaxis_title='Amplitude',
                    template='plotly_white')
plot_fft.first = True

In [ ]:
alpha = widgets.FloatSlider(
    value=0,
    min=-.5,
    max=.5,
    step=0.01,
    description='Alpha:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

In [ ]:
freqw = widgets.IntSlider(
    value=4000,
    min=200,
    max=8000,
    step=10,
    description='Freq:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [ ]:
lenw = widgets.Dropdown(
    options=['64', '96', '128', '160', '192'],
    value='64',
    description='Win Len:',
    disabled=False,
)

In [ ]:
controls = widgets.HBox([lenw, freqw, alpha])
widgets.VBox([fig, controls])

In [ ]:
alpha.observe(do_freping, names='value')
freqw.observe(do_freping, names='value')
lenw.observe(do_freping, names='value')
do_freping()